In [1]:
import os
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [2]:
# Deepseek
API_KEY = "sk-axxxxx"
ENDPOINT = "https://api.deepseek.com"
MODEL_NAME = "deepseek-chat"

llm = ChatOpenAI(
    base_url=ENDPOINT,
    model=MODEL_NAME,
    api_key=API_KEY,
    temperature=0,
)

In [ ]:
# Calling the llm
message = llm.invoke("repeat my last question")
print(message)

In [ ]:
# Setup the basic prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a helpful assistant. Respond in plain text, less than 30 words.",
        ),
        ("human", "{question}"),
    ]
)

# Chaining prompt and llm
chain = prompt | llm
message = chain.invoke({"question": "What is the name of imdb top 1 movie?"})
print(message)

In [7]:
# Store session histories
store = {}

# Function to get or create session history
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Added chat history in prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a helpful assistant. Respond in plain text, less than 30 words.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

# Chain prompt with LLM
chain = prompt | llm

# Runnable with message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [ ]:
message1 = with_message_history.invoke(
    {"question": "What is the name of imdb top 1 movie?"},
    config={"configurable": {"session_id": "abc123"}},
)
print(message1)

In [ ]:
message2 = with_message_history.invoke(
    {"question": "Which org created you?"},
    config={"configurable": {"session_id": "abc123"}},
)
print(message2)

In [ ]:
message3 = with_message_history.invoke(
    {"question": "What was my first question?"},
    config={"configurable": {"session_id": "abc123"}},
)
print(message3)

In [ ]:
print(store)